# Preamble

This notebook is split into stages that match those in the thesis document except for the pre-processing stage which is not included in the document and the CNN fine tuning stage which is separated out in the code. These stages are:

-  Stage 0 -- Pre-process Image Files
-  Stage 1 -- Image Preparation
-  Stage 2 -- Feature Extraction using a Convolutional Neural Network
-  Stage 3 -- Tile Clustering and Selection
-  Stage 4 -- Machine Learning 
-  Stage 5 -- Inter-Imageset Test
-  Stage 6 -- CNN Fine Tuning

Each stage is split into two main sections:
-  Functions
-  Interactive steps

Step through cells in order. Some interactive cells require settings to be specified (e.g. setting paths, image indexes, options selection). 

Many cells run into tens of minutes depending on number of images processed. Particularly long running cells (>2 hours) are noted. 

# Import Modules

In [45]:
from glob import glob
from os import path
from time import strftime
from timeit import timeit
from IPython.display import display
from pprint import pprint
import pickle
from subprocess import call
import re

from matplotlib import pyplot as plot
from matplotlib import cm
from matplotlib import pylab as pyl

import numpy as np
import pandas as pd

import openslide
from openslide import open_slide
import cv2
from PIL import Image

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import svm
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from keras.applications import ResNet50
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model

from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils import generic_utils
from keras.models import load_model

In [46]:
# Print all function for pd.DataFrames

def print_all(df):
    pd.set_option('display.max_rows', df.shape[0])
    print(df)
    pd.reset_option('display.max_rows')

# Stage 0: Pre-process Image Files

## Global Initialisation

In [47]:
#This step sets baselines globally

levels = [1]
l = 1

global images_root_dir
images_root_dir = '/Volumes/2T_HD/Masters_Project/images/'

#Create directories if they don't already exist
for l in levels:
    print('Creating level:', l)
    call(['mkdir', images_root_dir + 'level%s/' % (l, )])

global results_dir
results_dir = '/Volumes/2T_HD/Masters_Project/results/'

Creating level: 1


## Set Paths to SVS Files and Directories

In [4]:
# This cell specifies source locations of files and directories
# Source Astro SVS files
CBTC_A = '/Volumes/2T_HD/Masters_Project/astrocytoma/*.svs'
# Source Oligo SVS files (must be different to CBTC_A)
CBTC_O = '/Volumes/2T_HD/Masters_Project/oligodendroglioma/*.svs'
# Source TCGA raw images (flat directory)
TCGA_raw_data_dir = '/Volumes/2T_HD/Data/'
# Destination directory for creating patient directories including SVS images
TCGA_dirs = '/Volumes/2T_HD/Masters_Project/data/'
# Path to Patient_info.txt
patient_info = "/Volumes/2T_HD/Masters_Project/data/Patient_info.txt"

## Files

### View SVS File Characteristics

In [41]:
# This cell lists svs files
astrocytomafiles = glob(CBTC_A)
oligodendrogliomafiles = glob(CBTC_O)

astrocytomaslides = [open_slide(astrocytomafiles[i]) for i, file in enumerate(astrocytomafiles)]
oligodendrogliomaslides = [open_slide(oligodendrogliomafiles[i]) for i, file in enumerate(oligodendrogliomafiles)]

display(astrocytomaslides)
display(oligodendrogliomaslides)

[OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_1.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_11.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_14.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_15.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_17.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_18.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_19.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_2.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_23.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_26.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_28.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_29.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/astrocytoma/cbtc_train_31.svs'),
 OpenSlide('/V

[OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_10.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_12.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_13.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_16.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_20.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_21.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_22.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_24.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_25.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_27.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_3.svs'),
 OpenSlide('/Volumes/2T_HD/Masters_Project/oligodendroglioma/cbtc_train_30.svs'),
 OpenSlide('/Volu

In [39]:
# This cell displays downsampling of slides at each level, starting wth level 0 as 1.0. 
astrocytomaslides_level_downsamples = [astrocytomaslides[i].level_downsamples for i, slide in 
                                      enumerate(astrocytomaslides)]
oligodendrogliomaslides_level_downsamples = [oligodendrogliomaslides[i].level_downsamples for i, slide in 
                                      enumerate(oligodendrogliomaslides)]

display(astrocytomaslides_level_downsamples)
display(oligodendrogliomaslides_level_downsamples)

[(1.0, 4.000028173775849, 16.002179314742996, 32.00435862948599),
 (1.0, 4.000069571000547, 16.00117485976003, 64.01211424822071),
 (1.0, 4.0, 16.002581909316767),
 (1.0, 4.000068941744226, 16.001186876215765, 64.02816296010536),
 (1.0, 4.00007743534149, 16.001882558059016, 32.0064345634358),
 (1.0, 4.00005319424818, 16.00148742924302, 32.006363656785176),
 (1.0, 4.000035595783126, 16.00109764613604, 64.01967956008633),
 (1.0, 4.000058374771664, 16.00079118051236, 64.02391912590514),
 (1.0, 4.0, 16.000333611342786, 32.003336670003335),
 (1.0, 4.0003734217030935, 16.007471288878207),
 (1.0, 4.000083402835696, 16.00444997616996),
 (1.0, 4.0000236933137465, 16.001232227488153, 32.00549867273416),
 (1.0, 4.000117992712584, 16.002019692623826, 32.006763427003975),
 (1.0, 4.000176366843034, 16.0016462841016, 32.006927279697976),
 (1.0, 4.0000260728998285, 16.000943782487116, 64.01713159319587),
 (1.0, 4.000202922077922, 16.003130528891397)]

[(1.0, 4.00011320814337, 16.002067749092355, 32.01099055430055),
 (1.0, 4.000076769537847, 16.001535626535627, 32.003071253071255),
 (1.0, 4.000100408327198, 16.00200856989823, 32.00401713979646),
 (1.0, 4.000069319284625, 16.00138657792568, 32.00277315585136),
 (1.0, 4.000170419891039, 16.00205486798714, 32.007824348972534),
 (1.0, 4.000018601190476, 16.00077838223463, 32.00719106304494),
 (1.0, 4.00002707825616, 16.001682062045788, 32.006406382824494),
 (1.0, 4.000053009152914, 16.002745332219398, 32.005490664438796),
 (1.0, 4.000095008867494, 16.000380035469977, 32.004815002534215),
 (1.0, 4.000151255419986, 16.002942081502987),
 (1.0, 4.000122414004162, 16.00146914789422, 32.00548039937917),
 (1.0, 4.000271251705843, 8.000542503411687),
 (1.0, 4.000048631036327, 16.002016032553826, 64.0080641302153),
 (1.0, 4.000101927009087, 16.00323958015013, 32.00853276247733),
 (1.0, 4.000368007850835, 8.001554346307234),
 (1.0, 4.00007059155725, 16.00102956481193, 64.01301744798997)]

In [40]:
# This cell displays dimensions of slides on each level
astrocytomaslides_level_dimensions = [astrocytomaslides[i].level_dimensions for i, slide in 
                                      enumerate(astrocytomaslides)]
oligodendrogliomaslides_level_dimensions = [oligodendrogliomaslides[i].level_dimensions for i, slide in 
                                      enumerate(oligodendrogliomaslides)]


display(astrocytomaslides_level_dimensions)
display(oligodendrogliomaslides_level_dimensions)

[((89640, 70989), (22410, 17747), (5602, 4436), (2801, 2218)),
 ((147723, 69077), (36930, 17269), (9232, 4317), (2308, 1079)),
 ((65228, 28836), (16307, 7209), (4076, 1802)),
 ((178024, 58022), (44506, 14505), (11126, 3626), (2781, 906)),
 ((95924, 51658), (23981, 12914), (5995, 3228), (2997, 1614)),
 ((74821, 75577), (18705, 18894), (4676, 4723), (2338, 2361)),
 ((165949, 84949), (41487, 21237), (10371, 5309), (2592, 1327)),
 ((176501, 85030), (44125, 21257), (11031, 5314), (2757, 1328)),
 ((95924, 23008), (23981, 5752), (5995, 1438), (2997, 719)),
 ((27887, 37919), (6971, 9479), (1742, 2369)),
 ((47962, 34524), (11990, 8631), (2997, 2157)),
 ((84413, 83008), (21103, 20752), (5275, 5188), (2637, 2594)),
 ((94006, 79535), (23501, 19883), (5875, 4970), (2937, 2485)),
 ((70448, 34023), (17612, 8505), (4403, 2126), (2201, 1063)),
 ((151560, 76709), (37890, 19177), (9472, 4794), (2368, 1198)),
 ((27608, 29571), (6902, 7392), (1725, 1848))]

[((84413, 67031), (21103, 16757), (5275, 4189), (2637, 2094)),
 ((80576, 78159), (20144, 19539), (5036, 4884), (2518, 2442)),
 ((59759, 51040), (14939, 12760), (3734, 3190), (1867, 1595)),
 ((119520, 57706), (29880, 14426), (7470, 3606), (3735, 1803)),
 ((71943, 68955), (17985, 17238), (4496, 4309), (2248, 2154)),
 ((107521, 45460), (26880, 11365), (6720, 2841), (3360, 1420)),
 ((73861, 84188), (18465, 21047), (4616, 5261), (2308, 2630)),
 ((113191, 42668), (28297, 10667), (7074, 2666), (3537, 1333)),
 ((111552, 63155), (27888, 15788), (6972, 3947), (3486, 1973)),
 ((41832, 39671), (10458, 9917), (2614, 2479)),
 ((100720, 32678), (25180, 8169), (6295, 2042), (3147, 1021)),
 ((26858, 16353), (6714, 4088), (3357, 2044)),
 ((146764, 82254), (36691, 20563), (9172, 5140), (2293, 1285)),
 ((124701, 46574), (31175, 11643), (7793, 2910), (3896, 1455)),
 ((16307, 19556), (4076, 4889), (2038, 2444)),
 ((172664, 84999), (43166, 21249), (10791, 5312), (2697, 1328))]

In [ ]:
# This cell displays SVS file properties Including original magnification
astrocytomaslides_properties = [astrocytomaslides[i].properties for i, slide in 
                                      enumerate(astrocytomaslides)]

oligodendrogliomaslides_properties = [oligodendrogliomaslides[i].properties for i, slide in 
                                      enumerate(oligodendrogliomaslides)]

display(astrocytomaslides_properties)
display(oligodendrogliomaslides_properties)

### TCGA: Copy Raw images into patient directories

In [10]:
#Copy TCGA images into directories for each patient. Skip this cell if this pre-processing has already been done.

dirs = glob('%s/*' % (TCGA_raw_data_dir,))
for d in dirs:
    file = glob('%s/*' % (d,))
    file_name = file[0]
    new_dir = file_name.split('/')[5][:12]
    dir_name = TCGA_dirs + new_dir
    
    parms = 'cp %s %s' % (file_name, dir_name)
    print('Creating directory', dir_name)
    call(['mkdir', dir_name])
    print('Copying svs file:', file_name, 'to:', dir_name)
    call(parms, shell=True)

KeyboardInterrupt: 

### Copy OpenSlide images to Tiff format

In [6]:
#Copy all images to "images" directory as tiled tiff image. Use naming required for pre-processing.

#CBTC images
astro_files = glob(CBTC_A)
oligo_files = glob(CBTC_O)

for l in levels:
    print('Working on level:', l)
    dest_dir = images_root_dir + 'level%s/' % (l, )

    for a in astro_files:  
        dest_file_name = 'img' + a.split('.')[0].split('/')[-1].split('_')[2]
        print('Image being copied:', dest_file_name)
        parms = 'time vips openslideload --level %s %s %s%s.tiff[tile,compression=lzw]' \
                % (l, a, dest_dir, dest_file_name)
        print(parms)
        call(parms, shell=True)
            
    for o in oligo_files:  
        dest_file_name = 'img' + o.split('.')[0].split('/')[-1].split('_')[2]
        print('Image being copied:', dest_file_name)
        parms = 'time vips openslideload --level %s %s %s%s.tiff[tile,compression=lzw]' \
                % (l, a, dest_dir, dest_file_name)
        print(parms)
        call(parms, shell=True)
    
#TCGA images
patient_dirs = glob('%s/TCGA*' % (TCGA_dirs,))

for p in patient_dirs:
    files = glob('%s/*.svs' % (p,))

    for l in levels:
        print('Working on level:', l)
        
        dest_dir = images_root_dir + 'level%s/' % (l, )
        count = 0
        for f in files:
            count += 1
            if count == 1:
                dest_file_name = 'img' + f.split('.')[0].split('/')[-1].split('-')[2]
                print('Image being copied:', dest_file_name)
            else:
                dest_file_name = 'img' + f.split('.')[0].split('/')[-1].split('-')[2] + 'b'
                print('Image being copied:', dest_file_name)
                
            parms = 'time vips openslideload --level %s %s %s%s.tiff[tile,compression=lzw]' \
            % (l, f, dest_dir, dest_file_name)
            
            print(parms)
            call(parms, shell=True)


Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0
Working on level: 0


KeyboardInterrupt: 

## Image Indexes

### CBTC Indexes

In [48]:
#CBTC Image indexes

astro1 = [1,2,7,8,9,11,14,15,17,18,19,23,26,28,29,31]
oligo1 = [3,4,5,6,10,12,13,16,20,21,22,24,25,27,30,32]
images1 = [1,2,7,8,9,11,14,15,17,18,19,23,26,28,29,31,3,4,5,6,10,12,13,16,20,21,22,24,25,27,30,32]

astro1 = [str(i) for i in astro1]
oligo1 = [str(i) for i in oligo1]
images1 = [str(i) for i in images1]

### TCGA Indexes

In [49]:
#Build TCGA new_image list based on actual images in directory

img_files = glob('%s/*.tiff' % (images_root_dir + 'level%s/' % (l, )))
new_images = []
for i in img_files:
    img_ = i.split('.')[0].split('/')[-1].split('img')[1]
    if "_" not in img_ and len(img_)>2:
        new_image = img_
        new_images.append(new_image)
pprint(new_images)

['4938',
 '4938b',
 '4941',
 '4941b',
 '4942',
 '4942b',
 '4943',
 '4943b',
 '4944',
 '4944b',
 '5390',
 '5390b',
 '7902',
 '7902b',
 '7855',
 '7855b',
 '7299',
 '5963',
 '5963b',
 '5854',
 '5854b',
 '7694',
 '7694b',
 '7687',
 '6407',
 '6407b',
 '5964',
 '5964b',
 '7884',
 '7884b',
 '7301',
 '7301b',
 '5393',
 '5393b',
 '8164',
 '6666',
 '6666b',
 '8012',
 '8012b',
 '6692',
 '7693',
 '7693b',
 '7304',
 '7304b',
 '5849',
 '5849b',
 '7856',
 '7856b',
 '7879',
 '7879b',
 '7620',
 '7620b',
 '5395',
 '5395b',
 '7471',
 '7471b',
 '8165',
 '6410',
 '6410b',
 '8108',
 '7641',
 '7641b',
 '6669',
 '6669b',
 '7858',
 '7858b',
 '7302',
 '7302b',
 '7690',
 '7309',
 '7309b',
 '7875',
 '7875b',
 '6404',
 '6404b',
 '7467',
 '7467b',
 '8167',
 'A6IZ',
 '7605',
 '7605b',
 '7470',
 '6408',
 '6408b',
 '7880',
 '7880b',
 '8019',
 '7860',
 '7860b',
 '8013',
 '8013b',
 '8168',
 'A6J1',
 '7854',
 '7854b',
 '7611',
 '7611b',
 'A6S2',
 '7477',
 'A5TS',
 '7643',
 '7643b',
 '6690',
 '6690b',
 '7688',
 '7688b',
 

In [50]:
#TCGA index of patients to cancer types based on Patient_info.txt (describing download from TCGA site)

patients = open(patient_info, "r")
lines = patients.read().split(' ')
# pprint(lines)

cancer_types = []
cancer_grades = []
patients = []
for line in lines:
    #Handle a couple of lines out of alignment
    if "TCGA-DU-6407" in line:
        phrases = line.split()
        patient = phrases[-7].split('-')[2]
        cancer_type = phrases[-5]
        cancer_grade = phrases[-4]
    elif "TCGA-HT-7603" in line:
        phrases = line.split()
        patient = phrases[-5].split('-')[2]
        cancer_type = phrases[-3]
        cancer_grade = phrases[-2]
    #Remainder of dataset
    elif "TCGA" in line:
        phrases = line.split()
        patient = phrases[-6].split('-')[2]
        cancer_type = phrases[-4]
        cancer_grade = phrases[-3]

        patients.append(patient)
        cancer_types.append(cancer_type)
        cancer_grades.append(cancer_grade)

#Convert to dataframe with all subtypes and grades included
cancer = [('patient', patients), ('cancer subtype', cancer_types), ('cancer grade', cancer_grades)]
cancer_df = pd.DataFrame.from_items(cancer)

oligo2_df = cancer_df.loc[cancer_df['cancer subtype'] == 'Oligodendroglioma']
astro2_df = cancer_df.loc[cancer_df['cancer subtype'] == 'Astrocytoma']
mixed_df = cancer_df.loc[cancer_df['cancer subtype'] == 'Oligoastrocytoma']

# Create index for subsets of images with G2 only
astro2_g2_df = astro2_df.loc[astro2_df['cancer grade'] == 'G2']
oligo2_g2_df = oligo2_df.loc[oligo2_df['cancer grade'] == 'G2']

# Update index to remove images not actually available
#Balance dataset between astro and oligo (52 images each)
oligo2_test_df = oligo2_df[oligo2_df['patient'].isin(new_images)][52:]
astro2_df = astro2_df[astro2_df['patient'].isin(new_images)]
oligo2_df = oligo2_df[oligo2_df['patient'].isin(new_images)][:52]
mixed_df = mixed_df[mixed_df['patient'].isin(new_images)]

astro2_g2_df = astro2_g2_df.loc[astro2_g2_df['patient'].isin(new_images)]
oligo2_g2_df = oligo2_g2_df.loc[oligo2_g2_df['patient'].isin(new_images)][:19]

#Build astro2a, oligo2a, mixed2a lists
astro2a = astro2_df['patient'].tolist()
oligo2a = oligo2_df['patient'].tolist()
oligo2a_test = oligo2_test_df['patient'].tolist()
mixed2a = mixed_df['patient'].tolist()

astro2a_g2 = astro2_g2_df['patient'].tolist()
astro2a_test_g2 = astro2_g2_df['patient'].tolist()
oligo2a_g2 = oligo2_g2_df['patient'].tolist()

images2a = astro2a + oligo2a
images2a_g2 = astro2a_g2 + oligo2a_g2

images = images1 + images2a

#Produce index for second image for each patient (note there is not always a second image)
astro2b = []
oligo2b = []
#astro2a reflects patients - add 'b' to get second image for that patient
for i,j in zip(astro2a,oligo2a):
    ab = i + 'b'
    ob = j + 'b'
    astro2b.append(ab)
    oligo2b.append(ob)

#Select only files that exist on disk (some patients only have 1 file)
astro2b_df = pd.DataFrame(astro2b)
astro2b_df = astro2b_df[astro2b_df.isin(new_images)]
#Balance dataset between astro and oligo (38 "b" images each)
astro2b = astro2b_df.dropna()[0].tolist()[:38]

oligo2b_df = pd.DataFrame(oligo2b)
oligo2b_df = oligo2b_df[oligo2b_df.isin(new_images)]
oligo2b = oligo2b_df.dropna()[0].tolist()

images2b = astro2b + oligo2b

#Display indices
display(len(astro1))
display(len(oligo1))
display(len(images))

display(len(astro2a))
display(len(oligo2a))
display(len(images2a))

display(len(astro2b))
display(len(oligo2b))
display(len(images2b))

display(len(mixed2a))

# These indices may be varied for processing depending on need.
# For example the following was used to omit images with too few tiles that would cause an error in clustering:
# E.g. images2a = [i for i in images2a if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

16

16

136

52

52

104

38

38

76

43

In [13]:
#Chack balance of TCGA dataset
display(oligo2_df.loc[oligo2_df['cancer grade'] == 'G1'].shape)
display(oligo2_df.loc[oligo2_df['cancer grade'] == 'G2'].shape)
display(oligo2_df.loc[oligo2_df['cancer grade'] == 'G3'].shape)
display(astro2_df.loc[astro2_df['cancer grade'] == 'G1'].shape)
display(astro2_df.loc[astro2_df['cancer grade'] == 'G2'].shape)
display(astro2_df.loc[astro2_df['cancer grade'] == 'G3'].shape)
display(astro2_df.shape)
display(oligo2_df.shape)
display(astro2b_df.shape)
display(oligo2b_df.shape)

(0, 3)

(33, 3)

(19, 3)

(0, 3)

(17, 3)

(35, 3)

(52, 3)

(52, 3)

(52, 1)

(52, 1)

# Stage 1: Image Preparation

## Functions

In [177]:
#This function removes background from each image -- and does the writing to disk of "img-fg" 
#(original tiles unchanged)

def remove_background(img, img_to_read, img_to_write, l):  

    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    #Read
    print('Reading image', img, 'level', l)
    img_base = cv2.imread(img_to_read)
    
    print('Masking background..')
    #Downsize to level3 to derive mask
    scale = {1:8, 2:2, 3:1, 4:.5}
    dim = (int(img_base.shape[1]/scale[l]), int(img_base.shape[0]/scale[l]))
    img_l3 = cv2.resize(img_base, dim, interpolation = cv2.INTER_AREA)
    img_l3 = cv2.cvtColor(img_l3, cv2.COLOR_BGR2GRAY)
    
    #Remove background using Thresholding and Floodfill
    _, img_l3_threshold = cv2.threshold(img_l3, 220, 255, cv2.THRESH_BINARY_INV)
    mask = np.zeros((img_l3_threshold.shape[0]+2, img_l3_threshold.shape[1]+2), np.uint8)
    img_l3_floodfill = img_l3_threshold.copy()
    cv2.floodFill(img_l3_floodfill, mask, (0,0), 255)
    img_l3_floodfill_inverted = cv2.bitwise_not(img_l3_floodfill)
    img_l3_fg = img_l3_threshold | img_l3_floodfill_inverted
    
    #Check Image sizes
    d1 = img_base.shape[1] - int(img_l3_fg.shape[1]*scale[l]) 
    d0 = img_base.shape[0] - int(img_l3_fg.shape[0]*scale[l])
    
    #Upsize mask to fit base image
    mask_dim = (int(img_l3_fg.shape[1]*scale[l])+d1, int(img_l3_fg.shape[0]*scale[l])+d0)
    mask_fg = cv2.resize(img_l3_fg, mask_dim, interpolation = cv2.INTER_AREA)
    mask_fg = np.repeat(mask_fg[:, :, np.newaxis], 3, axis=2)
    
    #Apply foreground mask to base image
    img_fg = cv2.bitwise_and(img_base, mask_fg)
    
    #Print Image sizes
    display(mask_fg.shape)
    display(img_fg.shape)
    display(d1)
    display(d0)
    
    #Write
    print('Writing Foreground Image', img_to_write, 'for level:', l)
    cv2.imwrite(img_to_write, img_fg)
    
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
    
    return

In [178]:
# This function calculates the amount of tissue per tile

def calculate_tissue_percentage(tile):
    
    #Sample background and iterate through image colours
    img = Image.open(tile)
    background = img.getpixel((0,0))
    width, height = img.size
    background_amount = next(count for count,colour in img.getcolors(width*height) if colour==background)
    
    #Calculations
    tissue_amount = width*height - background_amount
    tissue_percent = tissue_amount*100.0/width/height
    
    return(tissue_percent)

In [179]:
# This function determines the tile numbers to use -- and does the wrting to disk of "tile_array"

def prepare_tiles(img, files, files_fg, tile_array_to_write):

    print('Selecting tiles with large tissue proportion for Image:', img)
    print(len(files), 'tiles being processed for image', img, 'on level', l)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    tile_list = []
    for file, file_fg in zip(files, files_fg):
        #Use tile only if it contains at least 80% tissue
        tissue_percent = calculate_tissue_percentage(file_fg)
        if l == 1:
            if tissue_percent >= 80:
                #Load tiles, preprocess and stack into an array
                tile = load_img(file, target_size=(224, 224))
                tile = img_to_array(tile)
                tile = preprocess_input(tile)
                tile_list.append(tile) 
        elif l == 2:
            if tissue_percent >= 60:
                #Load tiles, preprocess and stack into an array
                tile = load_img(file, target_size=(224, 224))
                tile = img_to_array(tile)
                tile = preprocess_input(tile)
                tile_list.append(tile)
        elif l == 3:
                #Load tiles, preprocess and stack into an array
                tile = load_img(file, target_size=(224, 224))
                tile = img_to_array(tile)
                tile = preprocess_input(tile)
                tile_list.append(tile)
    tile_array = np.stack(tile_list, axis=0)
    print(tile_array.shape[0], 'tiles selected from image', img, 'on level', l)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    #Save tile array for this image to disk
    print('Saving selected tiles for Image:', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    np.save(tile_array_to_write, tile_array)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
        
    return

## Interactive Image Preparation Steps

### Remove Background

In [272]:
#This cell saves to disk background-removed tiles "img-fg" -- used to select tiles for processing

#Select desired index (currently set to original CBTC images)
for i in images:
    print('Working on level:', l)

    for l in levels:
        img = 'img%s' % (i,)
        img_to_read = images_root_dir + 'level%s/img%s.tiff' % (l, i)
        img_to_write = images_root_dir + 'level%s/img_fg%s.tiff' % (l, i)
        remove_background(img, img_to_read, img_to_write, l)

Working on level: 1
Start time:  2017-06-03 08:13:55
Reading image img5396b level 1
Masking background..


(4399, 7378, 3)

(4399, 7378, 3)

2

7

Writing Foreground Image /Volumes/2T_HD/Masters_Project/images/level1/img_fg5396b.tiff for level: 1
End time:  2017-06-03 08:14:06
_________________________________________


### Perform interactive tiling of images

In [274]:
#This cell uses vips dzsave to tile images and copy them into a subdirectory: images_root_dir/level/0/imagename_files

for l in levels:
    print('Working on level:', l)
    dir = images_root_dir + 'level%s/' % (l, )
    
    for i in images:
        
        parms1 = 'time vips dzsave %simg%s.tiff %simg%s --depth one --tile-size 224 --overlap 0 --suffix .jpg' % (dir, i, dir, i)
        parms2 = 'time vips dzsave %simg_fg%s.tiff %simg_fg%s --depth one --tile-size 224 --overlap 0 --suffix .jpg' % (dir, i, dir, i)
        
        print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
        print(parms1)
        call(parms1, shell=True)
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        
        print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
        print(parms2)
        call(parms2, shell=True)
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

Working on level: 1
Start time:  2017-06-03 08:15:18
time vips dzsave /Volumes/2T_HD/Masters_Project/images/level1/img5396b.tiff img5396b --depth one --tile-size 224 --overlap 0 --suffix .jpg
End time:  2017-06-03 08:15:18
Start time:  2017-06-03 08:15:18
time vips dzsave /Volumes/2T_HD/Masters_Project/images/level1/img_fg5396b.tiff /Volumes/2T_HD/Masters_Project/images/level1/img_fg5396b --depth one --tile-size 224 --overlap 0 --suffix .jpg
End time:  2017-06-03 08:15:18


### Prepare Tiles

In [275]:
# This cell lists tile jpegs for images, and calls functions that check tissue percentage, loads tile if greater 
# than 80% tissue, preprocesses the tile, and saves all tiles for image to disk in a numpy array "tile_array". 
# It also prints out the number of tiles in the image which is important if the number is smaller than number 
# to be selected in later stages (e.g. clustering) for further procesing

for l in levels:
    print('Working on level:', l)
    
    for i in images:
        #Set names
        img = 'img%s' % (i,)
        dir = images_root_dir + 'level%s/img%s_files' % (l, i)
        files = glob('%s/0/*.jpg' % (dir,))
        dir_fg = images_root_dir + 'level%s/img_fg%s_files' % (l, i)
        files_fg = glob('%s/0/*.jpg' % (dir_fg,))
        tile_array_to_write = images_root_dir + 'level%s/img%s_files/tile_array' % (l, i)
        prepare_tiles(img, files, files_fg, tile_array_to_write)

Working on level: 1
Selecting tiles with large tissue proportion for Image: img5396b
660 tiles being processed for image img5396b on level 1
Start time:  2017-06-03 08:16:07
427 tiles selected from image img5396b on level 1
End time:  2017-06-03 08:16:21
Saving selected tiles for Image: img5396b
Start time:  2017-06-03 08:16:21
End time:  2017-06-03 08:16:25
_________________________________________


# Stage 2: Feature Extraction using a Convolutional Neural Network

## Functions

In [30]:
# This function writes to disk the "tile_features" array -- including features extracted from ResNet50 for each tile.

def extract_features(img, tile_array_to_read, features_to_write):
    
    #Extract features for this image from top layer of Resnet50 model
    print('Extracting features for Image:', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    tile_array = np.load(tile_array_to_read)
    tile_features = ResNet50_model.predict(tile_array)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    #Save features to disk
    print('Saving features for Image:', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    np.save(features_to_write, tile_features)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')bl
        
    return


In [ ]:
#Gather tuned features
# cnn_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
# tuned_features = cnn_model.predict(X_astro)

## Interactive CNN Steps

### Load ResNet Model

In [52]:
# Download Resnet50
ResNet50_model = ResNet50(weights="imagenet", include_top=False)

In [11]:
# View ReNet50 layers
for i, layer in enumerate(ResNet50_model.layers):
   print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 ac

### Extract Features

In [271]:
# This step is very long running (8+ hours).

# This step calls a function that extracts features from ResNet50 for each tile -- and writes 4D feature tensor 
# to disk in the "tile_features" numpy array. 
# Since no training is involved, all features can be extracted and saved ahead of time to allow for 
# quicker machine learning runs. 

for l in levels:
    print('Working on level:', l)
    
    for i in images:
        
        img = 'img%s' % (i,)
        tile_array_to_read = images_root_dir + 'level%s/img%s_files/tile_array.npy' % (l, i)
        features_to_write = images_root_dir + 'level%s/img%s_files/tile_features' % (l, i)
        
        if not path.exists(features_to_write + '.npy'):
            print('Image features extracted from ResNet50 will be written to:', features_to_write + '.npy')
            extract_features(img, tile_array_to_read, features_to_write)
        else:
            print(img, 'already processed')

Working on level: 1
img4938b already processed
img4941b already processed
img4942b already processed
img4943b already processed
img4944b already processed
img5393b already processed
img5394b already processed
img5397b already processed
img6188b already processed
img6666b already processed
img5854b already processed
img6402b already processed
img6405b already processed
img7010b already processed
img7013b already processed
img7298b already processed
img5963b already processed
img6688b already processed
img6689b already processed
img6691b already processed
img7476b already processed
img7478b already processed
img7479b already processed
img7485b already processed
img7601b already processed
img7606b already processed
img7680b already processed
img7686b already processed
img7691b already processed
img7854b already processed
img7855b already processed
img7857b already processed
img7858b already processed
img7860b already processed
img7884b already processed
img8011b already processed
img8104b

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/2T_HD/Masters_Project/images/level1/img5396b_files/tile_array.npy'

# Stage 3: Tile Clustering and Selection

## Functions

In [24]:
# This function performs SVD and calculates retained variance -- and writes to disk of "U", "s", "V", "V_k"

def perform_SVD(img, features_to_read, K, U_to_write, s_to_write, V_to_write, V_k_to_write):
    
    #Load saved Features
    print('Loading Saved Features for Image', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    tile_features = np.load(features_to_read)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        
    #PCA
    print('Starting PCA for Features of Image', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    U, s, V  = np.linalg.svd(tile_features[:,0,0,:])
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    #Retained Variance
    sum_evals = sum(s)
    print(sum_evals)
    retained_variance = [(i / sum_evals)*100 for i in s]
    cum_rv = np.cumsum(retained_variance)
    k_values = [1, 2, 5, 10, 20, 100]
    
    [print('For', k, 'features, SVD retained variance is:', str(int(cum_rv[k])) + '%') for k in k_values]
    
    #Create reduced matrix V_k
    V_k = V[:,:K]
    
#     print('Features shape',tile_features.shape)
#     print('tile features V_k',tile_features.dot(V_k).shape)
#     print('V shape',V.shape)
#     print('U shape',U.shape)
#     print('s shape',s.shape)
#     print('V_k shape',V_k.shape)
    
    #Save matrices to disk
    print('Saving Decomposed Matrices for Features of Image', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    np.save(U_to_write, U)
    np.save(s_to_write, s)
    np.save(V_to_write, V)
    np.save(V_k_to_write, V_k)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
    
    return

In [210]:
#This function saves to disk the "closest_tiles" array per image serving as an index of closest tiles to be used. 

def perform_feature_clustering(img, n_clusters, n_closest_tiles, X, closest_tiles_to_write, \
    remove_top_cluster, keep_only_top_cluster, remove_largest_clusters, new_tile_features_to_write):
    
    #Clustering
    print('Starting cluster analysis for features of image', img)
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    labels = kmeans.predict(X)
    centers = kmeans.cluster_centers_
    silhouette_avg = silhouette_score(X, labels)
    
    print("For number of clusters =", n_clusters,
          "The silhouette score is :", silhouette_avg)
    
    cluster_silhouettes_averages = []
    closest_tiles_list = []
    cluster_sizes = []
    
    for i in range(n_clusters):
        
        closest, _ = pairwise_distances_argmin_min(centers[[i]], X)
        #Tile index that would sort in ascending order of distances (minimum first)
        closest_tiles_array = np.argsort(pairwise_distances(centers, X))[i, :n_closest_tiles]
        #Silhouette average of all sihouettes within cluster
        cluster_silhouette_avg = np.mean(silhouette_samples(X, labels)[labels == i])
        
        closest_tiles_list.append(closest_tiles_array)
        cluster_silhouettes_averages.append(cluster_silhouette_avg)
        
        cluster_sizes.append(X[labels == i].shape[0])
    
    closest_tiles = np.stack(closest_tiles_list, axis=0)
    
    top_cluster_index = np.argmax(cluster_silhouettes_averages)
    worst_cluster_index = np.argmin(cluster_silhouettes_averages)
    largest_cluster_index = np.argmax(cluster_sizes)
    second_largest_cluster_index = np.argsort(cluster_sizes)[-2]

    print('The', n_clusters, 'ave cluster silhouettes are:', cluster_silhouettes_averages)
    print('')
    print('The best cluster is:', top_cluster_index)
    print('The worst cluster is:', worst_cluster_index)
    print('The largest cluster is:', np.argmax(cluster_sizes))
    print('')
    print('Closest tiles for best cluster', top_cluster_index, ':', closest_tiles[top_cluster_index])
    print('Closest tiles for worst cluster', worst_cluster_index, ':', closest_tiles[worst_cluster_index])
    print('Closest tiles for largest cluster', np.argmax(cluster_sizes), ':', closest_tiles[largest_cluster_index])
    
    for i in range(n_clusters):
            print('Number of tiles in cluster', i, 'is:', X[labels == i].shape[0]) #<-- size of cluster
    
    if remove_top_cluster == 'yes':
#         closest_tiles = np.delete(closest_tiles, top_cluster_index, 0)
#         print('Closest tiles new shape:', closest_tiles.shape)
        new_tiles_features = X[labels != top_cluster_index]
        np.save(new_tile_features_to_write, new_tile_features)
        print('Removing tiles from the top cluster...')
        
    elif keep_only_top_cluster == 'yes':
#         closest_tiles = closest_tiles[top_cluster_index]
#         print('Closest tiles new shape:', closest_tiles.shape)
        new_tiles_features = X[labels == top_cluster_index]
        np.save(new_tile_features_to_write, new_tile_features)
        print('Removing tiles from all but the top cluster...')
        
    elif remove_largest_clusters == 'yes':
        new_tile_features = X[labels != largest_cluster_index]
#         new_tile_features = new_tile_features[labels != second_largest_cluster_index]
        print(X.shape)
        print(new_tile_features.shape)
        np.save(new_tile_features_to_write, new_tile_features)
        print('Removing tiles from largest cluster...')
        
    print('Closest tiles new shape:', closest_tiles.shape)
    
    #Save closest tiles per cluster to disk
    print('Saving closest tiles per cluster for Features of Image', img)
    np.save(closest_tiles_to_write, closest_tiles)
    
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
      
    return(kmeans, labels, centers, silhouette_avg, cluster_silhouettes_averages, closest_tiles, \
           top_cluster_index)

## Interactive Clustering Steps

### Principal Component Analysis

In [72]:
# This cell calls a function that derives and writes to disk "V_k" (and U, s, V) array including (K) features 
# for clustering

for l in levels:
    print('Working on level:', l)
    
    #Set n_features
    K = 100
    
    for i in images:
        #Set names
        img = 'img%s' % (i,)
        features_to_read = images_root_dir + 'level%s/img%s_files/tile_features.npy' % (l, i)
        U_to_write = images_root_dir + 'level%s/img%s_files/U' % (l, i)
        s_to_write = images_root_dir + 'level%s/img%s_files/s' % (l, i)
        V_to_write = images_root_dir + 'level%s/img%s_files/V' % (l, i)
        V_k_to_write = images_root_dir + 'level%s/img%s_files/V_k' % (l, i)
        perform_SVD(img, features_to_read, K, U_to_write, s_to_write, V_to_write, V_k_to_write)
    

Working on level: 1
Loading Saved Features for Image img6186
Start time:  2017-05-24 11:26:35
End time:  2017-05-24 11:26:35
Starting PCA for Features of Image img6186
Start time:  2017-05-24 11:26:35
End time:  2017-05-24 11:26:37
1297.00545118
For 1 features, SVD retained variance is: 64%
For 2 features, SVD retained variance is: 67%
For 5 features, SVD retained variance is: 70%
For 10 features, SVD retained variance is: 73%
For 20 features, SVD retained variance is: 77%
For 100 features, SVD retained variance is: 89%
Saving Decomposed Matrices for Features of Image img6186
Start time:  2017-05-24 11:26:37
End time:  2017-05-24 11:26:38
_________________________________________
Loading Saved Features for Image img5851
Start time:  2017-05-24 11:26:38
End time:  2017-05-24 11:26:39
Starting PCA for Features of Image img5851
Start time:  2017-05-24 11:26:39
End time:  2017-05-24 11:26:40
685.852786474
For 1 features, SVD retained variance is: 68%
For 2 features, SVD retained variance i

### Cluster Exploration

In [109]:
# This cell builds silhouette diagrams of clusters for images, for different values of n_clusters.
# It also creates scatter plots over the feature space of the first two principal components

imlist = [4]
for i in imlist:
    img = 'img%s' % (i,)
    print('Image:', img)
    features_to_read = images_root_dir + 'level%s/img%s_files/tile_features.npy' % (l, i)
    tile_features = np.load(features_to_read)
    X = tile_features[:,0,0,:]
#     X = tile_features


    for n_clusters in range(10, 11):
        f, (graph1, graph2) = plot.subplots(1, 2)
        f.set_size_inches(30, 15)

        graph1.set_xlim([-0.1, 1])
        graph1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
        labels = kmeans.predict(X)
        V_to_read = images_root_dir + 'level%s/img%s_files/V.npy' % (l, i)
        V = np.load(V_to_read)
        V = V[:,:2]

        silhouette_avg = silhouette_score(X, labels)
        
        print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

        # Silhouette scores
        sample_silhouette_values = silhouette_samples(X, labels)

        y_lower = 10
        for i in range(n_clusters):
            cluster_silhouette_values = sample_silhouette_values[labels == i]
            cluster_silhouette_values.sort()
            cluster_size = cluster_silhouette_values.shape[0]
            
            y_upper = y_lower + cluster_size

            color = cm.spectral(float(i) / n_clusters)
            graph1.fill_betweenx(np.arange(y_lower, y_upper),0, cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

            graph1.text(-0.05, y_lower + 0.5 * cluster_size, str(i))

            y_lower = y_upper + 10

        graph1.set_title('Silhouette plot for %s clusters' % (n_clusters, ))
        graph1.set_xlabel('Silhouette scores')
        graph1.set_ylabel('Cluster')

        graph1.axvline(x=silhouette_avg, color="red", linestyle="--")

        graph1.set_yticks([])
        graph1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # Plot of clusters against features
        graph2.set_xlim([-0.01, .01])
#         graph2.set_ylim([-0.01, .01])
        graph2.set_xticks([-0.009, -0.006, -0.003, 0, 0.003, 0.006, 0.009])
#         graph2.set_yticks([-0.09, -0.06, -0.03, 0, 0.03, 0.06, 0.09])
        colors = cm.spectral(labels.astype(float) / n_clusters)
        graph2.scatter(V[:, 0], V[:, 1], marker='.', s=30, lw=0, alpha=0.7, c=colors)

        # Label the clusters
        centers = kmeans.cluster_centers_
        graph2.scatter(centers[:, 0], centers[:, 1], marker='o', c="white", alpha=1, s=200)

        for i, c in enumerate(centers):
            graph2.scatter(c[0], c[1], marker='$%d$' % (i, ), alpha=1, s=50)

        graph2.set_title('Clustered data visualisation')
        graph2.set_xlabel('Feature space for the 1st component')
        graph2.set_ylabel('Feature space for the 2nd component')

        plot.suptitle(('Silhouette analysis for KMeans clustering on sample data with %s clusters' % n_clusters), fontsize=14, fontweight='bold')
        
        plot.show()
#         pyl.savefig(img + str(n_clusters) + '.png', bbox_inches='tight')

Image: img4
For n_clusters = 10 The average silhouette_score is : 0.208102


ValueError: You passed in an invalid linestyle, `__`.  See docs of Line2D.set_linestyle for valid values.

### Cluster tiles and Calculate closest tiles to cluster centroid

In [211]:
# This cell calls a function  to perform kmeans clustering and distance to centroid calculations
# It writes the "closest_tiles" numpy array to disk which is an of n_clusters * n_closest_tiles index
# Options may be passed in to specify different clustering behaviour such as: remove top or largest cluster or
# recluster based removing a cluster on a previous clustering run.
# It also prints out information for further exploration of clustering

pca = 'no'
remove_top_cluster = 'no'
keep_only_top_cluster = 'no'
remove_largest_clusters = 'no'
subsequent_pass = 'no'

for l in levels:
    print('Working on level:', l)
    
    #Set clustering values
    n_closest_tiles = 10
    n_clusters = 10
    
    closest_tiles_list = []
    top_cluster_tiles_index_list = []
    for i in images:
        #Set names
        img = 'img%s' % (i,)
        closest_tiles_to_write = images_root_dir + 'level%s/img%s_files/closest_tiles' % (l, i)
        features_to_read = images_root_dir + 'level%s/img%s_files/tile_features.npy' % (l, i)
        new_tile_features_to_write = images_root_dir + 'level%s/img%s_files/new_tile_features' % (l, i)
        new_tile_features_to_read = images_root_dir + 'level%s/img%s_files/new_tile_features.npy' % (l, i)
        
        if pca == 'yes':
            V_k_to_read = images_root_dir + 'level%s/img%s_files/V_k.npy' % (l, i)
            V_k = np.load(V_k_to_read)
            tile_features_k = tile_features[:,0,0,:].dot(V_k)
            print(tile_features_k.shape)
            print(V_k.shape)
            X = tile_features_k
        else:
            print('Using full feature set')
            if subsequent_pass == 'yes':
                print('second clustering pass..')
                X = np.load(new_tile_features_to_read)
            else:
                tile_features = np.load(features_to_read)
                X = tile_features[:,0,0,:]
        
        kmeans, labels, centers, silhouette_avg, cluster_silhouettes_averages, closest_tiles, \
            top_cluster_tiles_index = perform_feature_clustering(img, n_clusters, \
            n_closest_tiles, X, closest_tiles_to_write, remove_top_cluster, keep_only_top_cluster, \
            remove_largest_clusters, new_tile_features_to_write)
        closest_tiles_list.append(closest_tiles)
        top_cluster_tiles_index_list.append(top_cluster_tiles_index)

Working on level: 1
Using full feature set
Starting cluster analysis for features of image img7687
Start time:  2017-05-27 22:10:10
For number of clusters = 10 The silhouette score is : 0.146571
The 10 ave cluster silhouettes are: [0.075919986, 0.014638377, 0.074325629, 0.20982367, 0.15851234, 0.18298297, -0.010670979, 0.10727489, 0.181798, 0.64596206]

The best cluster is: 9
The worst cluster is: 6
The largest cluster is: 7

Closest tiles for best cluster 9 : [1518 1368 1281  971 1371 1517 1433 1072  441  972]
Closest tiles for worst cluster 6 : [411 414 194 771 228 831  98  43 192 137]
Closest tiles for largest cluster 7 : [887 119 120 524 167 701 742 932 944 674]
Number of tiles in cluster 0 is: 45
Number of tiles in cluster 1 is: 31
Number of tiles in cluster 2 is: 239
Number of tiles in cluster 3 is: 132
Number of tiles in cluster 4 is: 175
Number of tiles in cluster 5 is: 239
Number of tiles in cluster 6 is: 110
Number of tiles in cluster 7 is: 254
Number of tiles in cluster 8 is

### View Tiles

In [676]:
# Interactive cell for viewing tiles of certain clusters
top_tiles = np.argsort(pairwise_distances(centers, X))[np.argmax(cluster_silhouettes_averages), :n_closest_tiles]
for t in top_tiles:
    winname = 'img%d' % (t,)
    print(winname)
    cv2.imshow(winname, tile_img2[t])

img2268
img2330
img818
img1474
img2415
img2117
img1143
img1198
img617
img569


In [677]:
cv2.destroyAllWindows()

### Select Random Tiles from Images

In [52]:
# This cell saves to disk the "random_tiles" array per image -- which is the equivalent to "closest_tiles" with 
# the index of tiles derived randomly rather than by clustering of feature vectors
# Later steps use either the "random_tiles" array or the "closest_tiles" array for testing

for l in levels:
    print('Working on level:', l)
    
    #Set clustering values
    n_random_tiles = 100
    
    for i in images:
        #Set names
        img = 'img%s' % (i,)
        random_tiles_to_write = images_root_dir + 'level%s/img%s_files/random_tiles' % (l, i)
        features_to_read = images_root_dir + 'level%s/img%s_files/tile_features.npy' % (l, i)
        tile_features = np.load(features_to_read)
        features_random_tiles = np.random.choice(tile_features.shape[0], n_random_tiles, replace=False)

        #Save closest tiles per cluster to disk
        print('Saving random tiles for Features of Image', img)
        print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
        np.save(random_tiles_to_write, features_random_tiles)
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        print('_________________________________________')


Working on level: 1
Saving random tiles for Features of Image img1
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img2
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img7
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img8
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img9
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img11
Start time:  2017-05-23 22:03:00
End time:  2017-05-23 22:03:00
_________________________________________
Saving random tiles for Features of Image img14
Start time:  

# Stage 4: Machine Learning

## Functions

### Tile Train/Test Harness

In [14]:
#This function reads feature vectors and index arrays, assembles the appropriate feature vectors and returns the array.

def build_feature_data(image_list, selection_type, pca):
        
#     print('Retrieve features from disk and create array')
    features_list = []
    for i in image_list:
        img = 'img%s' % (i,)
        features_to_read = images_root_dir + 'level%s/img%s_files/tile_features.npy' % (l, i)
        V_k_to_read = images_root_dir + 'level%s/img%s_files/V_k.npy' % (l, i)
        
        if selection_type == 'representative':
            tiles_to_read = images_root_dir + 'level%s/img%s_files/closest_tiles.npy' % (l, i)
        else:
            tiles_to_read = images_root_dir + 'level%s/img%s_files/random_tiles.npy' % (l, i)
            
        tiles = np.load(tiles_to_read)
        features = np.load(features_to_read)
        features = features[:,0,0,:]
        
        if pca == 'yes':
            V_k = np.load(V_k_to_read)
            features = features.dot(V_k)
            print('features shape', features.shape)
            
        features = np.array([features[tile] for _,tile in enumerate(tiles.flatten())])
        features_list.append(features)
    feature_data = np.array(features_list)
    
    return(feature_data)

In [15]:
# This function determines creates labels, concatenates data and returns X, y

def prepare_data(X_astro, X_oligo):

    X_astro = np.vstack(X_astro)
    X_oligo = np.vstack(X_oligo)
    
    y_astro = np.ones((X_astro.shape[0],), dtype=int)
    y_oligo = np.zeros((X_oligo.shape[0],), dtype=int)
    
    X = np.concatenate((X_astro, X_oligo), axis=0)
    print('Shape of X', X.shape)
    y = np.concatenate((y_astro, y_oligo), axis=0)
    print('Shape of y', y.shape)
    
    return(X, y)

### Classifier (Tile level)

In [16]:
# This function runs a grid search
def grid_cv(classifier, params, X_train, y_train):
    # Grid search and Cross validation
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    grid = GridSearchCV(classifier, params, scoring='accuracy', n_jobs=-1, cv=5, verbose=10)
    grid.fit(X_train, y_train)
    print(grid.best_params_)

    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
    return(grid)

In [17]:
# This function trains a model from grid search best estimator
def train_model_grid(grid, X_train, y_train):
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))

# Train tiles
    model = grid.best_estimator_
    model.fit(X_train, y_train)
    
    print('____________________________')
    print('grid', model)
    print('X_train', X_train.shape)
    print('y_train', y_train.shape)
    print('____________________________')

    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    print('_________________________________________')
    
    return(model)

In [18]:
# This function trains an SVM model, specifying parameters
def train_model(X_train, y_train):
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))

#Train tiles
    model = svm.SVC(C=1000, gamma=0.001, kernel='poly', max_iter=-1, probability=True)
    model.fit(X_train, y_train)
    
#     print('____________________________')
#     print('model', model)
#     print('X_train', X_train.shape)
#     print('y_train', y_train.shape)
#     print('____________________________')

    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
#     print('_________________________________________')
    
    return(model)

In [19]:
# This function makes predictions for tiles and determines accuracy independent of images involved
def test_model(model, X_test, y_test):
    #Print out tile predictions    
    model_predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, model_predictions)
#     print("Accuracy: %f\n" % accuracy_score(y_test, model_predictions))
#     print('Classification Report:')
#     print(classification_report(y_test, model_predictions))

    model_predictions_cm = confusion_matrix(y_test, model_predictions)
#     print(model_predictions_cm)
#     print('RMSE:',np.sqrt(mean_squared_error(y_test, model_predictions)))
#     print('R2 Score:',r2_score(y_test, model_predictions))

    return(accuracy)

### Image Level Test

In [20]:
# This function makes predictions for tiles for each image in the hold_out set and determines accuracies
def test_images(model, X_hold_out, hold_out_images, hold_out_astro):

    tile_proportion_correcta_list = []
    img_actuals = []
    img_predictions = []
    img_accuracies = []
    img_predictions_dict = {}
    img_accuracies_dict = {}
    
    for i,im in enumerate(hold_out_images):
        img = 'img%s' % (im,)
        tile_predictions = model.predict(X_hold_out[i])
        tile_probabilities = model.predict_proba(X_hold_out[i])
        tile_probabilities_sum_astro = np.sum(tile_probabilities[:,1])
        tile_probabilities_sum_oligo = np.sum(tile_probabilities[:,0])
        
        astro_count = np.sum(tile_predictions == 1)
        oligo_count = np.sum(tile_predictions == 0)
        
        img_actual = "".join(['Astro' if im in hold_out_astro else 'Oligo'])
        img_prediction = "".join(['Astro' if astro_count > oligo_count else 'Oligo'])
        img_accuracy = [astro_count/len(tile_predictions) if img_actual == 'Astro' else oligo_count/len(tile_predictions)]
        
        img_actuals.append(img_actual)
        img_predictions.append(img_prediction)
        img_accuracies.append(img_accuracy)
        img_accuracies_dict[im] = img_accuracy
        img_predictions_dict[im] = img_prediction

#Return arrays
    img_predictions = np.array(img_predictions)
    img_actuals = np.array(img_actuals)
    img_accuracies = np.array(img_accuracies)
    
    return(img_accuracies, img_predictions, img_actuals, img_accuracies_dict)

### Save / Load Results

In [29]:
# This function saves results objects to disk
def save_results(model, grid, fold_accuracies_runs, img_accuracies_table_runs, description):
    
    print('Saving results to disk for run')
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    print('Saving model')
    fileObject = open(run_dir + 'model_back', "wb")
    pickle.dump(model, fileObject)
    fileObject.close()
    
    print('Saving grid')
    fileObject = open(run_dir + 'grid_back', "wb")
    pickle.dump(grid, fileObject)
    fileObject.close()
    
    print('Saving fold_accuracies_runs')
    fileObject = open(run_dir + 'fold_accuracies_runs_back', "wb")
    pickle.dump(fold_accuracies_runs, fileObject)
    fileObject.close()
    
    print('Saving img_accuracies_table_runs')
    fileObject = open(run_dir + 'img_accuracies_table_runs_back', "wb")
    pickle.dump(img_accuracies_table_runs, fileObject)
    fileObject.close()
    
    print('Saving description')
    fileObject = open(run_dir + 'description', "wb")
    pickle.dump(description, fileObject)
    fileObject.close()

    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

    return

In [30]:
# This function loads results objects from disk
def load_results(run_dir):
    
    fileObject = open(run_dir + 'model_back', "rb")
    model_back = pickle.load(fileObject)
    fileObject.close()
    
    fileObject = open(run_dir + 'grid_back', "rb")
    grid_back = pickle.load(fileObject)
    fileObject.close()
    
    fileObject = open(run_dir + 'fold_accuracies_runs_back', "rb")
    fold_accuracies_runs_back = pickle.load(fileObject)
    fileObject.close()
    
    fileObject = open(run_dir + 'img_accuracies_table_runs_back', "rb")
    img_accuracies_table_runs_back = pickle.load(fileObject)
    fileObject.close()

    return(model_back, grid_back, fold_accuracies_runs_back, img_accuracies_table_runs_back)

## Interactive Cross Validation Testing

### Initialise

In [21]:
# This cell allows setting of a number of switches including classifier, parameters for grid search, random tile
# selection versus tile selection through clustering, use of PCA or not. These selections are used in if/else
# statements in other cells

#Switches----------------------------------------------------

#1. Classifier
# classifier_used = 'rf'
# classifier_used = 'svm'
classifier_used = 'xgb'

if classifier_used == 'svm':
    classifier = svm.SVC()
    svm_params = [{'kernel': ['poly', 'rbf'], 'gamma': [1e-3, 1e-4],
                      'C': [1e-2, 1e-1 ,1, 100, 1000]},
                 {'kernel': ['linear'], 'C': [1, 10, 100]}]
    params = svm_params

elif classifier_used == 'rf':
    classifier = RandomForestClassifier()
    rf_params = [{'n_estimators': [1000, 2000], 'max_depth': [None, 4],
                 'min_samples_split': [1, 2]}]
    params = rf_params
    
else:
    classifier = XGBClassifier()
    xgb_params = {
    'learning_rate':[.1,.2],
    'n_estimators':[200],
    'max_depth':[7],
    'gamma':[0,.01],
    'reg_alpha':[.1,.3],
    'reg_lambda':[.3,.5],
    'objective':['binary:logistic']
    }
    
    params = xgb_params
    
# grid = GridSearchCV(gbm, param_grid, scoring='accuracy', n_jobs=-1, verbose=10, cv=5)
# grid.fit(X_train, y_train)
    
#2. Test tiles selection type
selection_type = 'representative'
# selection_type = 'random'

#3. Run new grid search?
# use_gridsearch_parms = 'no'
use_gridsearch_parms = 'yes'

#4. Use PCA? Only applicable is clustering performed with PCA-reduced features
# pca = 'yes'
pca = 'no'

#Switches---------------------------------------------------

print('Classifier switch is:', classifier_used)
print('Test tile selection switch is:', selection_type)
print('Use grid determined paramters switch is:', use_gridsearch_parms)
print('PCA switch is:', pca)

Classifier switch is: xgb
Test tile selection switch is: representative
Use grid determined paramters switch is: yes
PCA switch is: no


### Set indexes for machine learning runs

In [20]:
# This cell used for setting indices of images for testing different machine learning scenarios


# Original CBTC image sets
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

astro = astro1
oligo = oligo1
images = astro1 + oligo1



# TCGA image sets
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# astro = astro2a
# oligo = oligo2a
# images = astro2a + oligo2a



# Explore removing poor images - CBTC
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# astro = [1,2,7,8,9,11,14,15,17,18,19,23,26,28,29,31]
# oligo = [3,4,5,6,10,12,13,16,20,21,22,24,25,27,30,32]
# images = [1,2,7,8,9,11,14,15,17,18,19,23,26,28,29,31,3,4,5,6,10,12,13,16,20,21,22,24,25,27,30,32]

# astro = [1,2,7,9,11,14,15,17,18,19,23,26,28]
# oligo = [3,4,5,6,10,13,16,24,25,27,30,32]
# images = [1,2,7,9,11,14,15,17,19,23,26,28,3,4,5,6,10,13,16,24,25,27,30,32]

# astro = [1,2,7,9,11,14,15,17,18,19,26,28]
# oligo = [3,4,5,6,10,12,16,20,24,25,27,30,32]

# astro = [8,23,29,31]
# oligo = [13,21,22,30]
# images = [8,23,29,31,13,21,22,30]



# Explore removing poor images - TCGA
# Use both Grade 2 and Grade 3 images
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# images = images1 + images2a
# #Remove small images with insufficient tiles (<100 tiles)
# images = [i for i in images if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

# astro = astro1 + astro2a
# astro = [i for i in astro if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

# oligo = oligo1 + oligo2a
# oligo = [i for i in oligo if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

# astro = [i for i in astro if i not in lt30]
# oligo = [i for i in oligo2a if i not in lt30]



# Use only Grade 2 images
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# images = images1 + images2a_g2
# images = [i for i in images if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

# astro = astro1 + astro2a_g2
# astro = [i for i in astro if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

# oligo = oligo1 + oligo2a_g2
# oligo = [i for i in oligo if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]



print('Number of astro images' ,len(astro))
print('Number of oligo images' ,len(oligo))
print('Total number of images' ,len(images))

Number of astro images 16
Number of oligo images 16
Total number of images 32


### Grid search

In [23]:
#Only run this cell once to run a new grid search (and set up train/test set if not done elsewhere). 

print('Running new grid search...')

#Build train data arrays based on image lists
print('Building feature data arrays...')
X_astro = build_feature_data(astro, selection_type, pca)
X_oligo = build_feature_data(oligo, selection_type, pca)

#Add labels and concatenate data
print('Adding labels and concatenating data...')
X, y = prepare_data(X_astro, X_oligo)

#Train/Test
print('Setting up train/test split...')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

#Run grid search up front on all images
print('Running grid search...')
grid = grid_cv(classifier, params, X_train, y_train)


Running new grid search...
Building feature data arrays...
Adding labels and concatenating data...
Shape of X (3200, 2048)
Shape of y (3200,)
Setting up train/test split...
Running grid search...
Start time:  2017-06-05 14:44:44
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV] reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.1min


[CV]  reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1, score=0.849609 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1, score=0.841797 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.835938 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.849609 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0,

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.1min


[CV]  reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1, score=0.837891 - 2.0min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1, score=0.845703 - 2.0min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1, score=0.851852 - 2.0min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.849609 - 2.0min
[CV] reg_alpha=0.1, n_estimators=200, gamma=0,

...

[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.845703 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.861598 - 2.2min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.843750 - 2.1min
[CV] reg_alpha=0.3, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.855469 - 2.1min
[CV] reg_alpha=0.3, n_est

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 13.8min


[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.2, score=0.835938 - 1.7min
[CV] reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.2 
[CV]  reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.3, max_depth=7, learning_rate=0.2, score=0.849903 - 1.8min
[CV] reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.2 
[CV]  reg_alpha=0.3, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.843750 - 2.1min
[CV] reg_alpha=0.1, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.2 
[CV]  reg_alpha=0.3, n_estimators=200, gamma=0.01, objective=binary:logistic, reg_lambda=0.5, max_depth=7, learning_rate=0.1, score=0.847656 - 2.1min
[CV] reg_alpha=0.1, n_est

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 18.9min finished


{'reg_alpha': 0.1, 'n_estimators': 200, 'gamma': 0, 'objective': 'binary:logistic', 'reg_lambda': 0.5, 'max_depth': 7, 'learning_rate': 0.2}
End time:  2017-06-05 15:04:52
_________________________________________


### Model Train and Test

In [24]:
# Build the model to determine baseline model accuracy at a tile level
print('Determining tile level accuracy...')
model = train_model_grid(grid, X_train, y_train)
print(model)

tile_accuracy = test_model(model, X_test, y_test)
print(tile_accuracy)

Determining tile level accuracy...
Start time:  2017-06-05 15:16:11
____________________________
grid XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=0.5,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
X_train (2560, 2048)
y_train (2560,)
____________________________
End time:  2017-06-05 15:17:26
_________________________________________
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=0.5,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
0.84375


### Image level cross validation testing

In [39]:
# Interactive K-fold Image cross validation
# Number f folds and runs is set. The hold out set is selected randomly and remaining images placed into train set. 
# Loops through all images in hold out set.
# Accuracies kept per image, per fold, per run

description = 'CBTC images, 4 folds, 4 runs, clustering, relabeled by removing 50/50 bets -- second set of astro'
run_dir = results_dir + '181/'

#Runs
runs = 4

# Select number of folds: 2, 4, 8, 16
folds = 4

print('Running with', selection_type, 'tiles...')

fold_accuracies_runs = []
fold_training_accuracies_runs = []
tile_fold_accuracies_runs = []
img_accuracies_table_runs = pd.DataFrame([])

for r in range(runs):
    
    fold_training_accuracies = []
    fold_accuracies = []
    tile_fold_accuracies = []
    img_accuracies_table = pd.DataFrame([])
    fold = 0
    print('RUN NUMBER...', r+1)
    
    # Train model for each set of images left after hold out images removed. 
    # Randomly select images to hold out.
    # Ensure an even number of astro and oligo images are chosen
    for i,j in zip(np.random.choice(astro, (folds, int(len(astro)/folds)), replace=False), np.random.choice(oligo, (folds, int(len(oligo)/folds)), replace=False)):
        
        hold_out_astro = i
        hold_out_oligo = j
        hold_out_images = np.hstack((hold_out_astro, hold_out_oligo))
        fold += 1
        
        # Determine Training images by removing hold_out images numbers from astro and oligo
        train_astro = [x for _,x in enumerate(astro) if x not in hold_out_astro]
        train_oligo = [x for _,x in enumerate(oligo) if x not in hold_out_oligo]
        train_images = np.hstack((train_astro, train_oligo))

        print('_________________________________________')
        print(' ')
        print('Fold...', fold)
        print('Train/Test Split:', (len(astro)-int(len(astro)/folds)+len(oligo)-int(len(oligo)/folds)), '/', (int(len(astro)/folds)+int(len(astro)/folds)))

        print(' ')
        print('Hold out images:', hold_out_images)
        print('Train images:', train_images)

        #Build train data arrays based on image lists
    #     print('Building feature data arrays...')
        X_astro = build_feature_data(train_astro, selection_type, pca)
        X_oligo = build_feature_data(train_oligo, selection_type, pca)

        #Add labels and concatenate data
    #     print('Adding labels and concatenating data...')
        X, y = prepare_data(X_astro, X_oligo)

        #Train/Test
    #     print('Setting up train/test split...')
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

        #Train model for this fold over training images
        print('')
        if use_gridsearch_parms == 'yes':
            print('Training tiles with grid search determined parameters with classifier',classifier_used,'...')
            model = train_model_grid(grid, X_train, y_train)
        else:
            print('Training tiles with statically defined parameters and classifier',classifier_used,'...')
            model = train_model(X_train, y_train)
            
        training_tile_accuracy = test_model(model, X_test, y_test)
        print('Tile level accuracy (training images):', training_tile_accuracy)
    #     print('_________________________________________')

        #Build test images
        X_hold_out = build_feature_data(hold_out_images, selection_type, pca)

        #Image level test
        img_accuracies, img_predictions, img_actuals, img_accuracies_dict = test_images(model, X_hold_out, hold_out_images, hold_out_astro)

        #Determine and print accuracy
        fold_accuracy = len(np.where(img_predictions == img_actuals)[0])/len(hold_out_images)
        fold_training_accuracies.append(training_tile_accuracy)
        fold_accuracies.append(fold_accuracy)
        tile_fold_accuracy = img_accuracies.mean()
        tile_fold_accuracies.append(tile_fold_accuracy)
        
        img_accuracies_col = pd.DataFrame.from_dict(img_accuracies_dict, orient='index')
        img_accuracies_col.columns = [r+1]
        img_accuracies_table = pd.concat([img_accuracies_table, img_accuracies_col]).sort_index()

    #     print('_________________________________________')
        print('')
        print('Labels for images:', img_actuals)
        print('Predictions for images:', img_predictions)
        print('Correct image predictions', len(np.where(img_predictions == img_actuals)[0]))
        print('')
        print('Tile level accuracies per image:', img_accuracies[:,0])
        print('Average Tile level Image accuracy:', tile_fold_accuracy)
        print('')
        print('Image level accuracy for fold:', fold_accuracy)
        print('Image level variance for fold:', img_accuracies.var())

    #Overall average accuracy and variance
    fold_accuracies = np.array(fold_accuracies)
    fold_training_accuracies = np.array(fold_training_accuracies)
    tile_fold_accuracies = np.array(tile_fold_accuracies)
    average_tile_fold_accuracies = tile_fold_accuracies.mean()
    average_fold_accuracies = fold_accuracies.mean()
    average_fold_training_accuracies = fold_training_accuracies.mean()
    tile_fold_accuracies_runs.append(average_tile_fold_accuracies)
    fold_accuracies_runs.append(average_fold_accuracies)
    fold_training_accuracies_runs.append(average_fold_training_accuracies)

    img_accuracies_table_runs = pd.concat([img_accuracies_table_runs, img_accuracies_table], axis=1).sort_index()

    print('_________________________________________')
    print('Image level test summary:')
    print('')
    for f in range(folds):
        print('Tile level accuracies for fold', f+1, ':',tile_fold_accuracies[f])
        print('Image level accuracies for fold', f+1, ':',fold_accuracies[f])
        print('')
        
    print('Average Tile Level Overall accuracy for Run:', tile_fold_accuracies.mean())
    print('')
    print('Average Image Level accuracy for Run:', fold_accuracies.mean())
    print('Overall variance for Run:', fold_accuracies.var())
    print('_________________________________________')

tile_fold_accuracies_runs = np.array(tile_fold_accuracies_runs)
fold_accuracies_runs = np.array(fold_accuracies_runs)
fold_training_accuracies_runs = np.array(fold_training_accuracies_runs)

for r in range(runs):
    print('Average tile level accuracy for Run', r+1, ':',tile_fold_accuracies_runs[r])
    print('Average Image Level accuracy for Run', r+1, ':',fold_accuracies_runs[r])
    print('')
print('')
print('Cross run Tile Level training accuracy for Run', r+1, ':',fold_training_accuracies_runs.mean())
print('Cross run Tile Level test accuracy:', tile_fold_accuracies_runs.mean())
print('')
print('_____________________________________________________________')
print('Cross run Image Level accuracy:', fold_accuracies_runs.mean())
print('Cross run Image Level variance:', fold_accuracies_runs.var())
print('_____________________________________________________________')
print('')
print('Cross run average accuracy per image:')
print(img_accuracies_table_runs.mean(axis=1))
print('')
print('Cross run average accuracy per astro image:')
print(img_accuracies_table_runs.loc[astro,:].mean(axis=1))
print('')
print('Cross run astro accuracy:', img_accuracies_table_runs.loc[astro,:].mean(axis=1).mean(axis=0))
print('Cross run astro variance:', img_accuracies_table_runs.loc[astro,:].mean(axis=1).var(axis=0))
print('')
print('Cross run average accuracy per oligo image:')
print(img_accuracies_table_runs.loc[oligo,:].mean(axis=1))
print('')
print('Cross run oligo accuracy:', img_accuracies_table_runs.loc[oligo,:].mean(axis=1).mean(axis=0))
print('Cross run oligo variance:', img_accuracies_table_runs.loc[oligo,:].mean(axis=1).var(axis=0))


#Save results

print('Saving results in', run_dir)
call(['mkdir', run_dir])
save_results(model, grid, fold_accuracies_runs, img_accuracies_table_runs, description)

Running with representative tiles...
RUN NUMBER... 1
_________________________________________
 
Fold... 1
Train/Test Split: 11 / 2
 
Hold out images: [14  4]
Train images: [ 7  8  9 11 12 17  3  5  6 13 30]
Shape of X (1100, 2048)
Shape of y (1100,)

Training tiles with grid search determined parameters with classifier xgb ...
Start time:  2017-06-05 18:35:37
____________________________
grid XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1, reg_lambda=0.5,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
X_train (880, 2048)
y_train (880,)
____________________________
End time:  2017-06-05 18:35:56
_________________________________________
Tile level accuracy (training images): 0.768181818182

Labels for images: ['Astro' 'Oligo']
Predictions for images: ['Astro' 'Olig

## Display Accuracies

In [417]:
lt30 = img_accuracies_table_runs.loc[img_accuracies_table_runs.mean(axis=1) <= .3]
display(lt30.shape)
display(img_accuracies_table_runs.loc[img_accuracies_table_runs.mean(axis=1).between(.3, .7, inclusive=True)].shape)
display(img_accuracies_table_runs.loc[img_accuracies_table_runs.mean(axis=1) >= .7].shape)

(9, 2)

(53, 2)

(44, 2)

# Stage 5: Inter-Imageset Test

## Set Indexes for Test Runs

In [238]:
# Subsets labelled for ease of testing

subset1_A = [1,2,7,9,11,14,15,17,18,19,26,28]
subset2_O = [3,4,5,6,10,12,16,20,24,25,27,30,32]
subset3_A = [8,23,29,31]
subset4_O = [13,21,22]

subset1_ATCGA = ['4938','4941','4942','4943','4944','6188','6290',\
         '6666','6691','6667','5854','7013','8158','A5TP','A5TU','A5TY','5963','6688','6689','A87N','7477','7478',\
         '7479','7485','7601','7680','7686','7691','7854','7855','7857','7858','7860','8011','8015','8104','8106',\
                 '8110']
               
subset2_OTCGA = ['5396','6668','5849','5874','6397',\
         '6399','6404','6410','7008','7015','7018','7294','7301','7302','7309','8164','8165','8168','A5TS','A6S8',\
         '6690','6692','A6IZ','A6J1','7469','7471','7472','7480','7616','7677','6400','A5TT']

subset3_ATCGA = [i for i in astro2a if i not in subset1_A]
subset4_OTCGA = [i for i in oligo2a if i not in subset2_O]

display(subset1_A)
display(subset2_O)
display(subset3_A)
display(subset4_O)
display(subset1_ATCGA)
display(subset2_OTCGA)
display(subset3_ATCGA)
display(subset4_OTCGA)

[1, 2, 7, 9, 11, 14, 15, 17, 18, 19, 26, 28]

[3, 4, 5, 6, 10, 12, 16, 20, 24, 25, 27, 30, 32]

[8, 23, 29, 31]

[13, 21, 22]

['4938',
 '4941',
 '4942',
 '4943',
 '4944',
 '6188',
 '6290',
 '6666',
 '6691',
 '6667',
 '5854',
 '7013',
 '8158',
 'A5TP',
 'A5TU',
 'A5TY',
 '5963',
 '6688',
 '6689',
 'A87N',
 '7477',
 '7478',
 '7479',
 '7485',
 '7601',
 '7680',
 '7686',
 '7691',
 '7854',
 '7855',
 '7857',
 '7858',
 '7860',
 '8011',
 '8015',
 '8104',
 '8106',
 '8110']

['5396',
 '6668',
 '5849',
 '5874',
 '6397',
 '6399',
 '6404',
 '6410',
 '7008',
 '7015',
 '7018',
 '7294',
 '7301',
 '7302',
 '7309',
 '8164',
 '8165',
 '8168',
 'A5TS',
 'A6S8',
 '6690',
 '6692',
 'A6IZ',
 'A6J1',
 '7469',
 '7471',
 '7472',
 '7480',
 '7616',
 '7677',
 '6400',
 'A5TT']

['4938',
 '4941',
 '4942',
 '4943',
 '4944',
 '6188',
 '6290',
 '6666',
 '6667',
 '5854',
 '6402',
 '6405',
 '7010',
 '7013',
 '7298',
 '7299',
 '8158',
 'A5TP',
 'A5TU',
 'A5TW',
 'A5TY',
 'A6S7',
 '5963',
 '6688',
 '6689',
 '6691',
 'A87N',
 '7476',
 '7477',
 '7478',
 '7479',
 '7485',
 '7601',
 '7606',
 '7607',
 '7680',
 '7686',
 '7691',
 '7854',
 '7855',
 '7857',
 '7858',
 '7860',
 '7884',
 '8011',
 '8015',
 '8104',
 '8106',
 '8110']

['5396',
 '6668',
 '6669',
 '5849',
 '5874',
 '6397',
 '6399',
 '6400',
 '6401',
 '6404',
 '6408',
 '6410',
 '7008',
 '7014',
 '7015',
 '7018',
 '7294',
 '7301',
 '7302',
 '7309',
 '8164',
 '8165',
 '8168',
 'A5TS',
 'A5TT',
 'A6S2',
 'A6S3',
 'A6S8',
 '5964',
 '6690',
 '6692',
 '7634',
 '7641',
 '8189',
 'A4MT',
 'A6IZ',
 'A6J1',
 '7467',
 '7469',
 '7470',
 '7471',
 '7472',
 '7480',
 '7481',
 '7602',
 '7605',
 '7616',
 '7620',
 '7677']

In [125]:
# Interactive Image test
# Allows training of TCGA set and testing against the CBTC set.
# Prints out predictions and accuracies for each image and overall for run

#Actuals
astro = astro1
oligo = oligo1
mixed = mixed2a

#Test
testing_images = images2a

#Train
train_astro = astro1
train_oligo = oligo1
train_images = np.hstack((train_astro, train_oligo))

#Run Directory to save cnn model
run_dir = results_dir + '200/'
print('Saving results in', run_dir)
call(['mkdir', run_dir])

print('_________________________________________')
print('Test images:', testing_images)
print('Train images:', train_images)
print(' ')
print('_________________________________________')


#Build train data arrays based on image lists
#     print('Building feature data arrays...')
X_astro = build_feature_data(train_astro, selection_type, pca)
X_oligo = build_feature_data(train_oligo, selection_type, pca)

#Add labels and concatenate data
#     print('Adding labels and concatenating data...')
X, y = prepare_data(X_astro, X_oligo)

#Train/Test for training images
print('Setting up train/test split for training images...')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

#Train model for this fold over training images
print('')

print('Training tiles with grid search determined parameters with classifier',classifier_used,'...')
model = train_model_grid(grid, X_train, y_train)

training_tile_accuracy = test_model(model, X_test, y_test)
print('Tile level accuracy (training images):', training_tile_accuracy)
print('_________________________________________')
print('Image Test Results...')

#Build test images
X_test = build_feature_data(testing_images, selection_type, pca)
print('x test', X_test.shape)

img_predictions = []
img_probabilities_astro = []
img_probabilities_oligo = []
astro_pred_count = 0
oligo_pred_count = 0 
astro_correct = 0
oligo_correct = 0 
astro_actual_count = 0 
oligo_actual_count = 0 
mixed_actual_count = 0 
unknown_actual_count = 0 
for i,im in enumerate(testing_images):
    img = 'img%s' % (im,)
    tile_predictions = model.predict(X_test[i])
    tile_probabilities = model.predict_proba(X_test[i])
    tile_probabilities_sum_astro = np.sum(tile_probabilities[:,1])
    tile_probabilities_sum_oligo = np.sum(tile_probabilities[:,0])

    astro_count = np.sum(tile_predictions == 1)
    oligo_count = np.sum(tile_predictions == 0)

    img_prediction = "".join(['Astro' if astro_count > oligo_count else 'Oligo'])
    
    if im in astro:
        img_actual = 'Astro'
        astro_actual_count += 1
    elif im in oligo:
        img_actual = 'Oligo'
        oligo_actual_count += 1
    elif im in mixed:
        img_actual = 'Oligoastro'
        mixed_actual_count += 1
    else:
        img_actual = 'Unknown'
        unknown_actual_count += 1

    print('For image:', img)
    print('The label is:', img_actual)
    print('The prediction is:', img_prediction)
    print('Probabilities of astro:', tile_probabilities_sum_astro)
    print('Probabilities of oligo:', tile_probabilities_sum_oligo)
    print('')

    if img_prediction == 'Astro':
        astro_pred_count += 1
        if img_prediction == img_actual:
            astro_correct += 1
    else:
        oligo_pred_count += 1
        if img_prediction == img_actual:
            oligo_correct += 1

    img_probabilities_astro.append(tile_probabilities_sum_astro)
    img_probabilities_oligo.append(tile_probabilities_sum_oligo)

print('_________________________________________')
# print('Accuracy:', (astro_correct + oligo_correct)*100/(astro_actual_count+oligo_actual_count))

print('Astro predictions:', astro_pred_count)

if test_images != mixed:
    print('Astro predictions correct:', astro_correct)
    print('Astro precision:', astro_correct/astro_pred_count)
    print('Astro recall:', astro_correct/astro_actual_count)
    print('Astro F1 Measure:', 2 * (astro_correct/astro_actual_count) * (astro_correct/astro_pred_count)\
          / ((astro_correct/astro_actual_count) + (astro_correct/astro_pred_count)))
print('')
print('Oligo predictions:', oligo_pred_count)

if test_images != mixed:
    print('Oligo predictions correct:', oligo_correct)
    print('Oligo precision:', oligo_correct/oligo_pred_count)
    print('Oligo recall:', oligo_correct/oligo_actual_count)
    print('Oligo F1 Measure:', 2 * (oligo_correct/oligo_actual_count) * (oligo_correct/oligo_pred_count)\
          / ((oligo_correct/oligo_actual_count) + (oligo_correct/oligo_pred_count)))
print('')
print('_________________________________________')
print('astro_probabilities', img_probabilities_astro)
print('oligo_probabilities', img_probabilities_oligo)


_________________________________________
Test images: ['6186', '5851', '5852', '5853', '5855', '5871', '5872', '6395', '6542', '7019', '7304', '7306', '8162', '8163', '8166', '8167', 'A5TR', 'A6S6', '5965', '7637', '7643', '8186', 'A4MU', 'A60K', 'A713', '7473', '7474', '7475', '7482', '7608', '7609', '7610', '7611', '7681', '7684', '7690', '7692', '7873', '7879', '7880', '7902', '8013', '8018']
Train images: [ 1  2  7  9 11 14 15 17 18 19 26 28 13 21 22]
 
_________________________________________
Shape of X (1500, 2048)
Shape of y (1500,)
Setting up train/test split for training images...

Training tiles with grid search determined parameters with classifier xgb ...
Start time:  2017-05-31 09:29:22
____________________________
grid XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0

### Load Objects from Results

In [157]:
run_dir = results_dir + '102/'
_, _, fold_accuracies_runs_back102, img_accuracies_table_runs_back102 = load_results(run_dir)

# Stage 6: CNN Fine Tuning

### Note on CNN Fine Tuning versus Transfer Learning
CNN Fine tuning requires a different approach to the structure of previous testing.
Mainly because you must take care of data leakage at the training stage.
With transfer learning it is possible to extract features from all tiles and keep them on disk.
With fine tuning, you must only train the network with images that will not be used to validate or test.
Therefore, CNN Fine Tuning is separated out from the rest of the code to simplify.

## Functions

In [41]:
def save_cnn_model(run_dir, model_name):
    run_dir = results_dir + '300/'
    print('Saving results in', run_dir)
    call(['mkdir', run_dir])
    print('Saving model')
    from keras.models import load_model

    model.save(model_name + '.h5') 
    
    return
    
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [42]:
# This function selects previously derived closest tiles based on clustered feature vectors from the non-fine tuned 
# second top layer of the ResNet50 model.
# It then builds the dataset for train/test

def build_data_for_cnn2(train_astro, train_oligo):
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    #Assemble clustered tiles for each image
    X_astro = []
    for i in train_astro:
        img = 'img%s' % (i,)
        tiles_to_read = images_root_dir + 'level%s/img%s_files/tile_array.npy' % (l, i)
        closest_tiles_to_read = images_root_dir + 'level%s/img%s_files/closest_tiles.npy' % (l, i)
        
        tiles = np.load(tiles_to_read)
        closest = np.load(closest_tiles_to_read)
        
        tiles = np.array([tiles[closest] for _,closest in enumerate(closest.flatten())])
        X_astro.append(tiles)
    X_astro = np.array(X_astro)
    X_astro = X_astro.reshape(-1,224,224,3)
    y_astro = np.ones((X_astro.shape[0],), dtype=int)
    
    X_oligo = []
    for i in train_oligo:
        img = 'img%s' % (i,)
        tiles_to_read = images_root_dir + 'level%s/img%s_files/tile_array.npy' % (l, i)
        closest_tiles_to_read = images_root_dir + 'level%s/img%s_files/closest_tiles.npy' % (l, i)
        
        tiles = np.load(tiles_to_read)
        closest = np.load(closest_tiles_to_read)
        
        tiles = np.array([tiles[closest] for _,closest in enumerate(closest.flatten())])
        X_oligo.append(tiles)
    X_oligo = np.array(X_oligo)
    X_oligo = X_oligo.reshape(-1,224,224,3)
    y_oligo = np.zeros((X_oligo.shape[0],), dtype=int)
    
    X = np.concatenate((X_astro, X_oligo), axis=0)
    print('Shape of X', X.shape)
    y = np.concatenate((y_astro, y_oligo), axis=0)
    print('Shape of y', y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    return(X_train, X_test, y_train, y_test, X, y)

In [43]:
# This function selects tiles randomly 
# It then builds the dataset for train/test

def build_data_for_cnn(train_astro, train_oligo):
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    #Assemble random tiles for each image
    X_astro = []
    for i in train_astro:
        img = 'img%s' % (i,)
        cnn_tiles_to_read = images_root_dir + 'level%s/img%s_files/cnn_random_tiles.npy' % (l, i)
        tiles = np.load(cnn_tiles_to_read)
        X_astro.append(tiles)
    X_astro = np.array(X_astro)
    X_astro = X_astro.reshape(-1,224,224,3)
    y_astro = np.ones((X_astro.shape[0],), dtype=int)

    X_oligo = []
    for i in train_oligo:
        img = 'img%s' % (i,)
        cnn_tiles_to_read = images_root_dir + 'level%s/img%s_files/cnn_random_tiles.npy' % (l, i)
        tiles = np.load(cnn_tiles_to_read)
        X_oligo.append(tiles)
    X_oligo = np.array(X_oligo)
    X_oligo = X_oligo.reshape(-1,224,224,3)
    y_oligo = np.zeros((X_oligo.shape[0],), dtype=int)

    X = np.concatenate((X_astro, X_oligo), axis=0)
    print('Shape of X', X.shape)
    y = np.concatenate((y_astro, y_oligo), axis=0)
    print('Shape of y', y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)
    print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
    
    return(X_train, X_test, y_train, y_test, X, y)

In [44]:
# This function tests hold out images against CNN trained model
# Prints out accuracies of images individually and overall as well as validated test score from CNN training

def test_cnn(hold_out_images, hold_out_astro, hold_out_oligo):
    print('Building test image batches...')
    X_test = []
    img_predictions = []
    img_actuals = []
    for i in hold_out_images:

        img = 'img%s' % (i,)
        cnn_tiles_to_read = images_root_dir + 'level%s/img%s_files/cnn_random_tiles.npy' % (l, i)
        X_test = np.load(cnn_tiles_to_read)
        X_test = X_test.reshape(-1,224,224,3)

        print('Predicting image:', img, '...')
        tile_predictions = model.predict(X_test)

        astro_count = np.sum(tile_predictions[:,1] >= .5)
        oligo_count = np.sum(tile_predictions[:,0] > .5)

        img_actual = "".join(['Astro' if i in hold_out_astro else 'Oligo'])
        img_prediction = "".join(['Astro' if astro_count > oligo_count else 'Oligo'])
        img_accuracy = [astro_count/X_test.shape[0] if img_actual == 'Astro' else oligo_count/X_test.shape[0]]
        print('Prediction:', img_prediction)
        print('Accuracy:', img_accuracy)

        img_accuracies.append(img_accuracy)
        img_predictions.append(img_prediction)
        img_actuals.append(img_actual)
        img_accuracies_dict[i] = img_accuracy

    #Determine and print accuracy
    fold_accuracy = np.sum(np.array(img_accuracies))/len(img_accuracies)

    print('_________________________________________')
    print('')
    print('Labels for images:', img_actuals)
    print('Predictions for images:', img_predictions)
    print('Correct image predictions', len(np.where(np.array(img_predictions) == np.array(img_actuals))[0]))
    print('')
    print('Image level accuracy for fold: %.2f%%' % (fold_accuracy*100,))

    print('Validation accuracy: %.2f%%' % (scores[1]*100),)


    #Save results
    print('Saving fold_accuracies_runs')
    call(['mkdir', run_dir])
    fileObject = open(run_dir + 'img_accuracies_dict%s' % (fold,), "wb")
    pickle.dump(img_accuracies_dict, fileObject)
    fileObject.close()
    
    return

## Interactive Steps

### Random Tile Selection

In [233]:
# This cell selects tiles randomly from each image and saves the to disk for using in CNN training and test.
# Preselecting tiles in this way saves some time but does not prevent CNN training runs being long running.

# These images have too few tiles so omit them (balance between astro and oligo)
images = [i for i in images2a if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]

for l in levels:
    print('Working on level:', l)
    
    #Set number of tiles
    n_random_tiles = 100
    
    print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
    for i in images:
        #Set names
        img = 'img%s' % (i,)
        random_tiles_to_write = images_root_dir + 'level%s/img%s_files/cnn_random_tiles' % (l, i)
        tiles_to_read = images_root_dir + 'level%s/img%s_files/tile_array.npy' % (l, i)
        tiles = np.load(tiles_to_read)
        idx = np.random.choice(tiles.shape[0], n_random_tiles, replace=False)
        tiles = tiles[idx]

        #Save closest tiles per cluster to disk
        print('Saving random tiles for Features of Image', img)
        np.save(random_tiles_to_write, tiles)
        
print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

Working on level: 1
Start time:  2017-06-04 16:23:07
Saving random tiles for Features of Image img4938b
Saving random tiles for Features of Image img4941b
Saving random tiles for Features of Image img4942b
Saving random tiles for Features of Image img4943b
Saving random tiles for Features of Image img4944b


ValueError: Cannot take a larger sample than population when 'replace=False'

### CNN Cross Validation

In [ ]:
# This step is very long running (2+ hours).

# Cross validation for tuned CNN
# Requires CNN to be trained against the training set with test images held out.
# Since CNN training is long running, testing with only 2 folds.

astro = astro1
oligo = oligo1
images = astro + oligo

#Interactive Image validation run

description = 'Train CNN cross validation on images1 using feature clustering from original feature vectors'
run_dir = results_dir + '700/'

#Runs
runs = 1


# Select number of folds: 2, 4, 8, 16
folds = 2

print('Running with CNN testing')

fold_accuracies_runs = []
fold_training_accuracies_runs = []
tile_fold_accuracies_runs = []
img_accuracies_table_runs = pd.DataFrame([])

for r in range(runs):
    img_actuals = []
    img_predictions = []
    img_accuracies = []
    img_predictions_dict = {}
    img_accuracies_dict = {}
        
    fold_training_accuracies = []
    fold_accuracies = []
    tile_fold_accuracies = []
    img_accuracies_table = pd.DataFrame([])
    fold = 0
    print('RUN NUMBER...', r+1)
    
    # Train model for each set of images left after hold out images removed. 
    # Randomly select images to hold out.
    # Ensure an even number of astro and oligo images are chosen
    for i,j in zip(np.random.choice(astro, (folds, int(len(astro)/folds)), replace=False), np.random.choice(oligo, (folds, int(len(oligo)/folds)), replace=False)):
        
        hold_out_astro = i
        hold_out_oligo = j
        hold_out_images = np.hstack((hold_out_astro, hold_out_oligo))
        fold += 1
        #Train image numbers: Mask hold_out image numbers from astro and oligo
        train_astro = [x for _,x in enumerate(astro) if x not in hold_out_astro]
        train_oligo = [x for _,x in enumerate(oligo) if x not in hold_out_oligo]
        train_images = np.hstack((train_astro, train_oligo))

        print('_________________________________________')
        print(' ')
        print('Fold...', fold)
        print('Train/Test Split:', (len(astro)-int(len(astro)/folds)+len(oligo)-int(len(oligo)/folds)), '/', (int(len(astro)/folds)+int(len(astro)/folds)))

        print(' ')
        print('Hold out images:', hold_out_images)
        print('Train images:', train_images)

        print('Building train data arrays...')
        X_train, X_test, y_train, y_test, _, _ = build_data_for_cnn2(train_astro, train_oligo)
        #Train cnn for this fold over training images
       
        
        print('_________________________________________')
        print(' ')
        
        print('Training ResNet50 top layer...')
        print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
        #Add averaging and dense layer, plus a logistic prediction layer to the base ResNet50 
        x = ResNet50_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(2, activation='softmax')(x)
        model = Model(inputs=ResNet50_model.input, outputs=predictions)
        
#         x = ResNet50_model.output
#         x.add(GlobalAveragePooling2D())
#         x.add(Dropout(0.5))
#         x.add(Dense(1024, activation='relu'))
#         predictions = Dense(2, activation='softmax')(x)
#         model = Model(inputs=ResNet50_model.input, outputs=predictions)

# Gather tuned features
# cnn_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
# tuned_features = cnn_model.predict(X_astro)

        for layer in ResNet50_model.layers:
            layer.trainable = False

        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

        #Scale and parameters
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)
        X_train /= 255
        X_test /= 255
        epochs = 1
        batch_size = 32

        #Train top layer
        top_layer_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
                         verbose=1, validation_data=(X_test, y_test))

        print('Saving results in', run_dir)
        call(['mkdir', run_dir])
        print('Saving model')
        model.save(run_dir + 'top_layer_model%s.h5' % (fold,)) 
    
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        print('_________________________________________')
        print(' ')
        
        print('Fine tuning ResNet50 with layers 49-50 ...')
        print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
        
        epochs = 2
        
        for layer in model.layers[:173]:
           layer.trainable = False
        for layer in model.layers[173:]:
           layer.trainable = True

        model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
        
        #Fine tune top 1 layer
        tuned_model_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
                     verbose=1, validation_data=(X_test, y_test))
    
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        
        print('Saving results in', run_dir)
        call(['mkdir', run_dir])
        print('Saving model')
        model.save(run_dir + 'tuned_model%s.h5' % (fold,))
        
        print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))
        print('_________________________________________')
        print(' ')
        
        scores = model.evaluate(X_test, y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

        print('_________________________________________')

        print('Building test image batches...')
        X_test = []
        img_predictions = []
        img_actuals = []
        tile_predictions_list = []
        for i in hold_out_images:

            img = 'img%s' % (i,)
            cnn_tiles_to_read = images_root_dir + 'level%s/img%s_files/cnn_random_tiles.npy' % (l, i)
            X_test = np.load(cnn_tiles_to_read)
            X_test = X_test.reshape(-1,224,224,3)

            print('Predicting image:', img)
            tile_predictions = model.predict(X_test)

            astro_count = np.sum(tile_predictions[:,1] >= .5)
            oligo_count = np.sum(tile_predictions[:,0] > .5)

            img_actual = "".join(['Astro' if i in hold_out_astro else 'Oligo'])
            img_prediction = "".join(['Astro' if astro_count > oligo_count else 'Oligo'])
            img_accuracy = [astro_count/X_test.shape[0] if img_actual == 'Astro' else oligo_count/X_test.shape[0]]
            print('Prediction:', img_prediction)
            print('Accuracy:', img_accuracy)
            
            tile_predictions_list.append(tile_predictions)
            img_accuracies.append(img_accuracy)
            img_predictions.append(img_prediction)
            img_actuals.append(img_actual)
            img_accuracies_dict[i] = img_accuracy

        #Determine and print accuracy
        fold_accuracy = np.sum(np.array(img_accuracies))/len(img_accuracies)

        print('_________________________________________')
        print('')
        print('Labels for images:', img_actuals)
        print('Predictions for images:', img_predictions)
        print('Correct image predictions', len(np.where(np.array(img_predictions) == np.array(img_actuals))[0]))
        print('')
        print('Image level accuracy for fold:', fold_accuracy)
        print('Validation accuracy: %.2f%%' % (scores[1]*100,))

        #Save results
        print('Saving img_accuracies_dict')
        call(['mkdir', run_dir])
        fileObject = open(run_dir + 'img_accuracies_dict%s' % (fold,), "wb")
        pickle.dump(img_accuracies_dict, fileObject)
        fileObject.close()
        
        print('Saving tile_predictions_list')
        call(['mkdir', run_dir])
        fileObject = open(run_dir + 'tile_predictions_list%s' % (fold,), "wb")
        pickle.dump(tile_predictions_list, fileObject)
        fileObject.close()

### CNN Test Inter-Image sets

In [34]:
# This step is very long running (3+ hours).

# Interactive Image test for CNN testing
# Allows training of TCGA set and testing against the CBTC set.
# Prints out predictions and accuracies for each image and overall for run

astro = [i for i in astro2a if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]
oligo = [i for i in oligo2a if i not in ['5393', '5394', '5397', '5390', '5395', '7676']]


print('Fine tuning with following images as input')
print(astro)
print(oligo)

description = 'Train CNN on TCGA images2a using feature clustering from original feature vectors'
run_dir = results_dir + '800/'


#Load data

X_train, X_test, y_train, y_test, X, y = build_data_for_cnn2(astro, oligo)

print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
#Add averaging and dense layer, plus a logistic prediction layer to the base ResNet50 
x = ResNet50_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=ResNet50_model.input, outputs=predictions)

for layer in ResNet50_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Scale and parameters
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)
X_train /= 255
X_test /= 255
epochs = 2
batch_size = 32

#Train top layer
top_layer_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
                 verbose=1, validation_data=(X_test, y_test))

print('Saving results in', run_dir)
call(['mkdir', run_dir])
print('Saving model')
model.save(run_dir + 'top_layer_model.h5')

print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

# Fine tune top few layers

print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
epochs = 2

for layer in model.layers[:169]:
   layer.trainable = False
for layer in model.layers[169:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

tuned_model_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,\
                 verbose=1, validation_data=(X_test, y_test))

print('Saving results in', run_dir)
call(['mkdir', run_dir])
print('Saving model')
model.save(run_dir + 'tuned_model.h5')
print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

print("Start time: ",strftime("%Y-%m-%d %H:%M:%S"))
test_cnn(images1, astro1, oligo1)
print("End time: ",strftime("%Y-%m-%d %H:%M:%S"))

Fine tuning with following images as input
['4938', '4941', '4942', '4943', '4944', '6188', '6290', '6666', '6667', '5854', '6402', '6405', '7010', '7013', '7298', '7299', '8158', 'A5TP', 'A5TU', 'A5TW', 'A5TY', 'A6S7', '5963', '6688', '6689', '6691', 'A87N', '7476', '7477', '7478', '7479', '7485', '7601', '7606', '7607', '7680', '7686', '7691', '7854', '7855', '7857', '7858', '7860', '7884', '8011', '8015', '8104', '8106', '8110']
['5396', '6668', '6669', '5849', '5874', '6397', '6399', '6400', '6401', '6404', '6408', '6410', '7008', '7014', '7015', '7018', '7294', '7301', '7302', '7309', '8164', '8165', '8168', 'A5TS', 'A5TT', 'A6S2', 'A6S3', 'A6S8', '5964', '6690', '6692', '7634', '7641', '8189', 'A4MT', 'A6IZ', 'A6J1', '7467', '7469', '7470', '7471', '7472', '7480', '7481', '7602', '7605', '7616', '7620', '7677']
Start time:  2017-06-04 23:42:00
Shape of X (9800, 224, 224, 3)
Shape of y (9800,)
End time:  2017-06-04 23:53:20
Start time:  2017-06-04 23:53:24
Train on 7840 samples, v